In [1]:
%matplotlib inline

In [18]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

import numpy as np
import pandas as pd
import tensorflow as tf
sess = tf.Session()

from keras import backend as K
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, Flatten, Dense
from keras.regularizers import l2
from keras.optimizers import SGD

from keras import backend as K
from keras.regularizers import Regularizer
from keras.objectives import kld

import matplotlib.pyplot as plt

# Stacked AE
TODO
- data generator for subsampled data for 4096 input

In [28]:
autoencoder = Sequential()
autoencoder.add(Flatten(input_shape=(1, 32, 32)))
autoencoder.add(Dense(100,activation='sigmoid'))
autoencoder.add(Dense(100,activation='sigmoid'))
autoencoder.add(Dense(4096,activation='sigmoid'))
autoencoder.summary()

sgd = SGD(lr=0.01, decay=1e-4, momentum=0.9, nesterov=True)
autoencoder.compile(loss='mean_squared_error', optimizer=sgd)

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
flatten_7 (Flatten)              (None, 1024)          0           flatten_input_6[0][0]            
____________________________________________________________________________________________________
dense_22 (Dense)                 (None, 100)           102500      flatten_7[0][0]                  
____________________________________________________________________________________________________
dense_23 (Dense)                 (None, 100)           10100       dense_22[0][0]                   
____________________________________________________________________________________________________
dense_24 (Dense)                 (None, 4096)          413696      dense_23[0][0]                   
Total params: 526296
______________________________________________________________________

# Pre-training of W4 (unsupervised)
- code for training
- code for saving weights
- save outputs

In [19]:
# For the Kullback-Leibler divergence regularization

class SparseActivityRegularizer(Regularizer):

    def __init__(self, l1=0., l2=0.,p = 0.05,beta = 3):
        self.l1 = K.cast_to_floatx(l1)
        self.l2 = K.cast_to_floatx(l2)
        self.uses_learning_phase = True
        self.layer = None
        self.p = p

    def set_layer(self, layer):
        if self.layer is not None:
            raise Exception('Regularizers cannot be reused')
        self.layer = layer

    def __call__(self, loss):
        if self.layer is None:
            raise Exception('Need to call `set_layer` on '
                            'ActivityRegularizer instance '
                            'before calling the instance.')
        regularized_loss = loss
        
        
        for i in range(len(self.layer.inbound_nodes)):
            output = self.layer.get_output_at(i)
            
            p_hat = K.mean(output, axis=0)
            regularized_loss += beta*K.sum(self.p * K.log(self.p / (p_hat+K.epsilon())))
            
        return K.in_train_phase(regularized_loss, loss)

    def get_config(self):
        return {'name': self.__class__.__name__,
                'l1': float(self.l1),
                'l2': float(self.l2)}


In [30]:
lambd = 3e-3
rho = 0.1
beta = 3

activity_regularizer = SparseActivityRegularizer(p=rho,beta=beta)

h1 = Sequential()
h1.add(Flatten(input_shape=(1, 64, 64)))
h1.add(Dense(100,activation='sigmoid',W_regularizer=l2(lambd/2),activity_regularizer=activity_regularizer))
h1.add(Dense(4096,W_regularizer=l2(lambd/2),activation='sigmoid'))

sgd = SGD(lr=0.01, decay=1e-4, momentum=0.9, nesterov=True)
h1.compile(loss='mean_squared_error', optimizer=sgd)

h1.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
flatten_9 (Flatten)              (None, 4096)          0           flatten_input_8[0][0]            
____________________________________________________________________________________________________
dense_26 (Dense)                 (None, 100)           409700      flatten_9[0][0]                  
____________________________________________________________________________________________________
dense_27 (Dense)                 (None, 4096)          413696      dense_26[0][0]                   
Total params: 823396
____________________________________________________________________________________________________


# Pre training of W5 (unsupervised)
- code for training
- code for saving weights
- save outputs

In [31]:
lambd = 3e-3
rho = 0.1
beta = 3

activity_regularizer = SparseActivityRegularizer(p=rho,beta=beta)

h1 = Sequential()
h1.add(Dense(100,activation='sigmoid',W_regularizer=l2(lambd/2),activity_regularizer=activity_regularizer,input_dim=100))
h1.add(Dense(100,activation='sigmoid',W_regularizer=l2(lambd/2))


sgd = SGD(lr=0.01, decay=1e-4, momentum=0.9, nesterov=True)
h1.compile(loss='mean_squared_error', optimizer=sgd)
h1.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
dense_28 (Dense)                 (None, 100)           10100       dense_input_6[0][0]              
____________________________________________________________________________________________________
dense_29 (Dense)                 (None, 100)           10100       dense_28[0][0]                   
Total params: 20200
____________________________________________________________________________________________________


# Pre training of W6
- code for training
- code for saving weights
- used with labeled data

In [ ]:
lambd = 3e-3
rho = 0.1
beta = 3

activity_regularizer = SparseActivityRegularizer(p=rho,beta=beta)

h1 = Sequential()
h1.add(Dense(4096,W_regularizer=l2(lambd/2),input_dim=100,activation='sigmoid'))

sgd = SGD(lr=0.01, decay=1e-4, momentum=0.9, nesterov=True)
h1.compile(loss='mean_squared_error', optimizer=sgd)
h1.summary()

# Fine-tuning of parameters
- load weights into layers below
- there is a 1/2 factor infront of the mse...?

In [ ]:
lambd = 1e-4

autoencoder = Sequential()
autoencoder.add(Flatten(input_shape=(1, 32, 32)))
autoencoder.add(Dense(100,activation='sigmoid',W_regularizer=l2(lambd/2)))
autoencoder.add(Dense(100,activation='sigmoid',W_regularizer=l2(lambd/2)))
autoencoder.add(Dense(4096,activation='sigmoid',W_regularizer=l2(lambd/2)))
autoencoder.summary()

sgd = SGD(lr=0.01, decay=1e-4, momentum=0.9, nesterov=True)
autoencoder.compile(loss='mean_squared_error', optimizer=sgd)